In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import astropy.coordinates as coord
from astropy.table import Table, vstack
from astropy.io import fits
import astropy.units as u

import gala.coordinates as gc

import pickle

coord.galactocentric_frame_defaults.set('v4.0');

In [3]:
plt.style.use('notebook')

In [4]:
fr = gc.GD1()

In [5]:
corners0 = np.array([[-80, -10], [-80,5], [10,5], [10,-10]])

In [6]:
for corners in [corners0]:
    c_corner = coord.SkyCoord(phi1=corners[:,0]*u.deg, phi2=corners[:,1]*u.deg, frame=fr)
    ceq_corner = c_corner.transform_to(coord.ICRS)
    q_base ='''SELECT * FROM gaiaedr3.gaia_source
WHERE parallax - 2*parallax_error < 0.5 AND
      CONTAINS(POINT('ICRS', ra, dec), 
               POLYGON('ICRS', 
                       {0.ra.degree}, {0.dec.degree}, 
                       {1.ra.degree}, {1.dec.degree}, 
                       {2.ra.degree}, {2.dec.degree}, 
                       {3.ra.degree}, {3.dec.degree})) = 1
'''
    print(q_base.format(ceq_corner[3], ceq_corner[2], ceq_corner[1], ceq_corner[0]))

SELECT * FROM gaiaedr3.gaia_source
WHERE parallax - 2*parallax_error < 0.5 AND
      CONTAINS(POINT('ICRS', ra, dec), 
               POLYGON('ICRS', 
                       218.66034239121856, 50.18564516484523, 
                       220.8639874172285, 65.14151546244803, 
                       121.55418932787543, 4.764630672532538, 
                       134.58828370034513, -2.6790431755558206)) = 1



## Combine Gaia tables

In [10]:
essential = ['bp_rp', 'phot_g_mean_mag', 'ra', 'dec', 'parallax', 'parallax_error',
             'pmra', 'pmdec', 'dec_error', 'pmra_error', 'pmdec_error']
tnames = ['gd1_{:d}.fits.gz'.format(x) for x in range(1,5)]

In [11]:
for e, tname in enumerate(tnames):
    t_ = Table(fits.getdata('/home/ana/projects/legacy/elz_reader/data/gaia/{:s}'.format(tname)))
    t_.keep_columns(essential)
    
    if e==0:
        tout = t_
    else:
        tout = vstack([tout, t_])

### Deredden

In [13]:
from dustmaps.sfd import SFDQuery
def deredden(t):
    # load SFD reader
    sfd = SFDQuery()
    
    # Gaia extinction coefficients from Babusiaux+2018
    kg = np.array([0.9761, -0.1704, 0.0086, 0.0011, -0.0438, 0.0013, 0.0099])
    kbp = np.array([1.1517, -0.0871, -0.0333, 0.0173, -0.0230, 0.0006, 0.0043])
    krp = np.array([0.6104, -0.0170, -0.0026, -0.0017, -0.0078, 0.00005, 0.0006])
    
    # query dust map
    c = coord.SkyCoord(ra=t['ra']*u.deg, dec=t['dec']*u.deg, frame='icrs')
    ebv = sfd(c)
    a0 = 3.1*ebv
    
    # 
    N = len(t)
    p = np.zeros((4,N))
    bp_rp = np.zeros(N)
    
    p[0] = a0 * (krp[3] - kbp[3])
    p[1] = a0 * (krp[2] - kbp[2])
    p[2] = a0 * (krp[1] - kbp[1]) + a0**2 * (krp[6] - kbp[6]) - 1
    p[3] = a0 * (krp[0] - kbp[0]) + a0**2 * (krp[4] - kbp[4]) + a0**3 * (krp[5] - kbp[5]) + t['bp_rp']
    
    for i in range(N):
        r = np.roots(p[:,i])
        ind = np.argmin(np.abs(r - t['bp_rp'][i]))
        bp_rp[i] = r[ind]
    
    ag = (kg[0] + kg[1]*bp_rp + kg[2]*bp_rp**2 + kg[3]*bp_rp**3 + kg[4]*a0 + kg[5]*a0**2 + kg[6]*bp_rp*a0) * a0
    g = t['phot_g_mean_mag'] - ag
    
    t['bp_rp0'] = bp_rp
    t['g0'] = g

In [14]:
ind = np.isfinite(tout['bp_rp']) & np.isfinite(tout['phot_g_mean_mag'])
tout = tout[ind]
print(len(tout))

3119112


In [15]:
deredden(tout)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:29: ComplexWarning: Casting complex values to real discards the imaginary part


In [16]:
tout.write('../data/gd1.fits.gz', overwrite=True)

### Distant, retrograde selection

In [19]:
ind_far = (tout['parallax'] - 2*tout['parallax_error']<0.5)
ind_retro = (tout['pmra']<0) & (tout['pmdec']<0)

In [20]:
np.sum(ind_far), np.sum(ind_retro), np.sum(ind_far & ind_retro)

(2728358, 1730395, 1530791)

In [21]:
ind = ind_far & ind_retro

In [22]:
tout2 = tout[ind]

In [23]:
tout2.write('../data/gd1_retro.fits.gz', overwrite=True)